# quiz 01

---

## step 00 : setting

- import module
- data path

In [1]:
import os
import sys
import pandas as pd
import numpy as np
import re
import requests

# self-definition package.module
import quiz.myfun01 as myfun

In [3]:
path_root = os.path.join(os.getcwd(), "../")
path_data = os.path.join(path_root, "./data")
path_result = os.path.join(path_root, "./result")

if not os.path.exists(path_result):
    os.mkdir(path_result)

---

## step 01 : download data

下載【內政部不動產時價登錄網】中，  
位於【臺北市/新北市/桃園市/臺中市/高雄市】的【不動產買賣】資料，  
請選擇【本期下載】。

- 手動下載
- 用 api 下載。


In [3]:
import requests 

In [4]:
csv_url = [
    'https://plvr.land.moi.gov.tw//Download?fileName=a_lvr_land_a.csv',
    'https://plvr.land.moi.gov.tw//Download?fileName=f_lvr_land_a.csv',
    'https://plvr.land.moi.gov.tw//Download?fileName=h_lvr_land_a.csv',
    'https://plvr.land.moi.gov.tw//Download?fileName=b_lvr_land_a.csv',
    'https://plvr.land.moi.gov.tw//Download?fileName=e_lvr_land_a.csv',
    'https://plvr.land.moi.gov.tw//Download?fileName=manifest.csv&q=aa,fa,ha,ba,ea&type=csv',
    'https://plvr.land.moi.gov.tw//Download?fileName=schema-build.csv&q=aa,fa,ha,ba,ea&type=csv',
    'https://plvr.land.moi.gov.tw//Download?fileName=schema-park.csv&q=aa,fa,ha,ba,ea&type=csv',
    'https://plvr.land.moi.gov.tw//Download?fileName=schema-main.csv&q=aa,fa,ha,ba,ea&type=csv',
    'https://plvr.land.moi.gov.tw//Download?fileName=schema-land.csv&q=aa,fa,ha,ba,ea&type=csv',
]

In [5]:
for i in range(len(csv_url)):
    
    # download
    req = requests.get(csv_url[i])
    url_content = req.content
    
    # filename
    fname = re.search("(?<=fileName=).+\.csv", csv_url[i]).group(0)
    path_fname = os.path.join(path_data, fname)
    print(path_fname)
    
    # save
    csv_file = open(path_fname, 'wb')
    csv_file.write(url_content)
    csv_file.close()

D:\JobHouseSearch\Job-Search-2020\company\cathay\exam\codePy\.././data\a_lvr_land_a.csv
D:\JobHouseSearch\Job-Search-2020\company\cathay\exam\codePy\.././data\f_lvr_land_a.csv
D:\JobHouseSearch\Job-Search-2020\company\cathay\exam\codePy\.././data\h_lvr_land_a.csv
D:\JobHouseSearch\Job-Search-2020\company\cathay\exam\codePy\.././data\b_lvr_land_a.csv
D:\JobHouseSearch\Job-Search-2020\company\cathay\exam\codePy\.././data\e_lvr_land_a.csv
D:\JobHouseSearch\Job-Search-2020\company\cathay\exam\codePy\.././data\manifest.csv
D:\JobHouseSearch\Job-Search-2020\company\cathay\exam\codePy\.././data\schema-build.csv
D:\JobHouseSearch\Job-Search-2020\company\cathay\exam\codePy\.././data\schema-park.csv
D:\JobHouseSearch\Job-Search-2020\company\cathay\exam\codePy\.././data\schema-main.csv
D:\JobHouseSearch\Job-Search-2020\company\cathay\exam\codePy\.././data\schema-land.csv



---

## step 02 : data

使用【pandas】套件，讀取檔名【a_lvr_land_a】【b_lvr_land_a】【e_lvr_land_a】【f_lvr_land_a】【h_lvr_land_a】五份資料集，建立 dataframe 物件【df_a】【df_b】【df_e】【df_f】【df_h】。

- read
- union

---

In [6]:
path_all_data = [os.path.join(path_data, files) for files in os.listdir(path_data) if files.startswith('_lvr_land', 1)]

In [7]:
'''
- row 1 is english column. remove it.
- check there are five dataframes: [x for x in dir() if 'df_' in x]
- can concat in for-loop. but I follow the steps.
'''
df_all_list = []
for i_path in path_all_data:
    print(i_path)
    name_data = 'df_' + os.path.basename(i_path).split('.')[0][0]
    tmp_data = pd.read_csv(i_path, encoding = 'utf-8')[1:]
    df_all_list.append(tmp_data)
    exec(name_data + " = tmp_data" )
    print(tmp_data.shape)
    print('-'*70)

D:\JobHouseSearch\Job-Search-2020\company\cathay\exam\codePy\.././data\a_lvr_land_a.csv
(955, 33)
----------------------------------------------------------------------
D:\JobHouseSearch\Job-Search-2020\company\cathay\exam\codePy\.././data\b_lvr_land_a.csv
(1652, 33)
----------------------------------------------------------------------
D:\JobHouseSearch\Job-Search-2020\company\cathay\exam\codePy\.././data\e_lvr_land_a.csv
(1716, 33)
----------------------------------------------------------------------
D:\JobHouseSearch\Job-Search-2020\company\cathay\exam\codePy\.././data\f_lvr_land_a.csv
(2397, 33)
----------------------------------------------------------------------
D:\JobHouseSearch\Job-Search-2020\company\cathay\exam\codePy\.././data\h_lvr_land_a.csv
(1628, 33)
----------------------------------------------------------------------


## step 03 : combine

操作 dataframe 物件，將五個物件合併成【df_all】

---

In [8]:
df_all = pd.concat(df_all_list, ignore_index = True)

## step 04 : filter and summary

以下列條件從【df_all】篩選計算出結果，並分別輸出【csv 檔案】

- filter_a.csv  
  - 【主要用途】為【住家用】
  - 【建物型態】為【住宅大樓】
  - 【總樓層數】需【大於等於十三層】

In [9]:
cond01 = (df_all['主要用途']=='住家用')

# df_all['建物型態'].unique()
cond02 = df_all['建物型態'].str.startswith('住宅大樓')

df_all['total_floor'] = df_all['總樓層數'].transform(myfun.chtnum2dignum)
# df_all['總樓層數'].unique()
# df_all['total_floor'].unique()
cond03 = df_all['total_floor'] >= 13

In [10]:
filter_a = df_all[cond01 & cond02 & cond03]
filter_a.to_csv(os.path.join(path_result, "./filter_a.csv"), index = 0, encoding = 'utf-8')

--

- filter_b.csv
  - 計算【總件數】
  - 計算【總車位數】(透過交易筆棟數)
  - 計算【平均總價元】
  - 計算【平均車位總價元】
  
<font color='red'><b>  
題目似乎沒有說，是依照全部(df_all)來做 filter_b.csv 的計算，抑或是依照上述結果(filter_a)繼續往下做。  
所以我就依照全部(df_all)繼續運算。如果是要按照上述，就把下方的 df_all 改為 filter_a 即可。  
</b></font>

In [11]:
print('總件數 : ', len(df_all.index))

df_all['車位數'] = df_all['交易筆棟數'].astype(str).str.extract('((?<=車位)[0-9]+)', expand = False)
print('總車位數 : ', df_all['車位數'].astype(int).sum())

df_all['總價元'].astype(float).mean()
print('平均總價元 : ', df_all['總價元'].astype(float).mean())

# 沒有車位，阿就不用納入計算阿!! (小陷阱XD)
print('平均車位總價元 : ', df_all.loc[df_all['車位數']!='0', '車位總價元'].astype(float).mean())

總件數 :  8348
總車位數 :  4143
平均總價元 :  15301387.960229995
平均車位總價元 :  995157.8910724312


In [12]:
attr = ['總件數', '總車位數', '平均總價元', '平均車位總價元']
value = [len(df_all.index), 
         df_all['車位數'].astype(int).sum(),
         df_all['總價元'].astype(float).mean(),
         df_all.loc[df_all['車位數']!='0', '車位總價元'].astype(float).mean()]
filter_b = pd.DataFrame.from_dict({'attr':attr, 'value':value})
filter_b.to_csv(os.path.join(path_result, "./filter_b.csv"), index = 0, encoding = 'utf-8', float_format = '%.2f')

## END

In [6]:
pd.read_csv(os.path.join(path_result, "./filter_a.csv"), nrows=5)

,鄉鎮市區,交易標的,土地區段位置建物區段門牌,土地移轉總面積平方公尺,都市土地使用分區,非都市土地使用分區,非都市土地使用編定,交易年月日,交易筆棟數,移轉層次,...,車位移轉總面積(平方公尺),車位總價元,備註,編號,主建物面積,附屬建物面積,陽台面積,電梯,移轉編號,total_floor
0,大同區,房地(土地+建物)+車位,民權西路121~150號,4.25,都市：其他:第三種商業區。,NaN,NaN,1100319,土地1建物1車位1,六層,...,0.00,0,陽台外推；夾層；其他增建；,RPOOMLQJNHGGFBA17DA,23.58,1.23,3.58,有,5.0,14
1,萬華區,房地(土地+建物)+車位,環河南路二段1~30號,8.62,都市：其他:第肆種商業區(依都市計畫說明書圖規定辦理，始得作第肆種商業區使用)(原屬第參種商...,NaN,NaN,1100414,土地2建物1車位1,四層,...,19.94,0,NaN,RPRPMLRJNHGGFBA67DA,51.91,6.01,3.26,有,6.0,24
2,內湖區,房地(土地+建物)+車位,行善路271~300號,49.87,都市：其他:住商混合區(為容積接收區)。,NaN,NaN,1100308,土地1建物1車位1,三層,...,32.13,3300000,NaN,RPPQMLMKLHGGFCA07DA,158.48,14.90,15.89,有,NaN,14
3,內湖區,房地(土地+建物)+車位,成功路四段167巷1~30號,33.79,都市：其他:第三種住宅區。,NaN,NaN,1100326,土地2建物1車位1,九層,...,41.04,2500000,其他增建；,RPWQMLQLLHGGFCA28DA,121.12,3.88,10.18,有,7.0,17
4,北投區,房地(土地+建物),中山路1~30號,13.74,都市：其他:特定觀光商業專用區。,NaN,NaN,1100402,土地1建物1車位0,七層,...,0.00,0,陽台外推；,RPUPMLNJNHGGFEA87DA,51.92,0.00,16.31,有,NaN,25


In [4]:
pd.read_csv(os.path.join(path_result, "./filter_b.csv"))

,attr,value
0,總件數,8348.00
1,總車位數,4143.00
2,平均總價元,15301387.96
3,平均車位總價元,995157.89
